## Setup and Data Import

In [1]:
import sys
sys.path.insert(0, '..')

from joblib import load

import Functions as fxns
from Functions import np, pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from datetime import timedelta

from sklearn.impute import SimpleImputer

In [2]:
# !python ../Preprocessing.py

In [3]:
claims = load('../claims.pkl')

## Claims DF

### New Columns

In [4]:
claims['AgeAtService'] = ((claims.ClaimStartDt - claims.DOB)
                              / timedelta(days=365)).astype(int)

claims['HasDied'] = claims.DOD.notna()
fxns.re_encode_bool(claims, ['HasDied'])

In [5]:
physician_cols = \
    claims.columns[claims.columns.str.contains('Physician')].to_list()
phys_count_cols  = ['HasAllPhysicians', 'HasAnyPhysicians', 'HasNoPhysicians']

claims['HasAllPhysicians'] = claims[physician_cols].notna().all(axis=1)
claims['HasAnyPhysicians'] = claims[physician_cols].notna().any(axis=1)
claims['HasNoPhysicians']  = claims[physician_cols].isna().all(axis=1)

fxns.re_encode_bool(claims, phys_count_cols)

In [6]:
claims['ClaimDuration'] = \
    (claims.ClaimEndDt  - claims.ClaimStartDt).dt.days + 1
claims['IPDuration']    = \
    (claims.DischargeDt - claims.AdmissionDt).dt.days + 1

In [7]:
claims['ClaimCost'] = \
    claims.InscClaimAmtReimbursed + claims.DeductibleAmtPaid
claims['PercInsCovered'] = \
    round((claims.InscClaimAmtReimbursed / claims.ClaimCost) * 100)
claims['DailyClaimCost'] = \
    round(claims.ClaimCost / claims.ClaimDuration)

In [8]:
claim_cols = claims.columns.to_list()
claim_cols.remove('Provider')

### Variables

In [9]:
# lists:
numeric_cols     = fxns.cols_by_dtype(claims)[0]
categorical_cols = fxns.cols_by_dtype(claims)[1]
date_cols        = fxns.cols_by_dtype(claims)[2]
physician_cols   = \
    claims.columns[claims.columns.str.contains('Physician')].to_list()
chronic_cols     = \
    claims.columns[claims.columns.str.contains('Chronic')].to_list()
diagnosis_cols   = \
    claims.columns[claims.columns.str.contains('Diagnosis')].to_list()
procedure_cols   = \
    claims.columns[claims.columns.str.contains('Procedure')].to_list()

# dataframes:
outpatient_claims = claims.loc[claims.IsOutpatient == 1]
inpatient_claims  = claims.loc[claims.IsOutpatient == 0]

## Providers DF

### Data Import

In [10]:
providers = claims.groupby(['PotentialFraud', 'Provider', 'IsOutpatient'
                           ]).size().reset_index().drop(0, axis=1)

In [11]:
lucas = pd.read_csv('./data/providers.csv')
lucas.columns = ['Provider', 'DualPatientType_Perc', 'HasNoPhys_Perc', 'HasTop5AdmtCode']

In [12]:
ryan = pd.read_csv('./data/Ryan_providers.csv')
ryan.drop('Unnamed: 0', axis=1, inplace=True)
ryan.rename(columns = {'Patient_Attphy_Ratio': 'BenesPerAttPhys',
                'Patient_Operphy_Ratio': 'BenesPerOperPhys',
                'Patient_Otherphy_Ratio': 'BenesPerOthPhys',
                'BeneID_Nunique_IP': 'IP_UniqueBenes_Count',
                'BeneID_Nunique_OP': 'OP_UniqueBenes_Count',
                'State_Nunique_IP': 'IP_UniqueSt_Count',
                'State_Nunique_OP': 'OP_UniqueSt_Count'}, inplace=True)

In [13]:
providers = providers.merge(lucas).merge(ryan)

### Beneficiaries

In [14]:
providers['GenderZero_Perc'] = claims.groupby(['Provider', 'IsOutpatient']).Gender.mean().values
providers['HasRenalDisease_Perc'] = claims.groupby(['Provider', 'IsOutpatient']).RenalDisease.mean().values
providers['HasDied_Perc'] = claims.groupby(['Provider', 'IsOutpatient']).HasDied.mean().values

In [15]:
for col in chronic_cols:
    colname = f'{col}_Perc'
    providers[colname] = claims.groupby(['Provider', 'IsOutpatient'])[col].mean().values

### Doctors

In [16]:
for col in phys_count_cols:
    colname = f'{col}_Perc'
    providers[colname] = claims.groupby(['Provider', 'IsOutpatient'])[col].mean().values

### Money

In [19]:
money_cols = \
    ['InscClaimAmtReimbursed', 'DeductibleAmtPaid',
     'NoOfMonths_PartACov', 'NoOfMonths_PartBCov', 'AgeAtService',
     'HasDied', 'PercInsCovered',
     'HasAllPhysicians', 'HasAnyPhysicians', 'HasNoPhysicians',
     'ClaimDuration', 'IPDuration', 'ClaimCost', 'DailyClaimCost',
     'IPAnnualReimbursementAmt', 'IPAnnualDeductibleAmt',
     'OPAnnualReimbursementAmt', 'OPAnnualDeductibleAmt']

In [20]:
for col in money_cols:
    colname = f'{col}_Perc'
    providers[colname] = claims.groupby(['Provider', 'IsOutpatient'])[col].mean().values

## MISC

In [21]:
providers.head(12)

,PotentialFraud,Provider,IsOutpatient,DualPatientType_Perc,HasNoPhys_Perc,HasTop5AdmtCode,BenesPerAttPhys,BenesPerOperPhys,BenesPerOthPhys,Att_Phy_Mult,Oper_Phy_Mult,Other_Phy_Mult,IP_Dup_Perc,OP_Dup_Perc,OP_No_Diag_Perc,IP_No_Proc_Perc,In_Top5_St_Perc,IP_UniqueBenes_Count,IP_UniqueSt_Count,OP_UniqueBenes_Count,OP_UniqueSt_Count,GenderZero_Perc,HasRenalDisease_Perc,HasDied_Perc,Alzheimers_Chronic_Perc,HeartFailure_Chronic_Perc,KidneyDisease_Chronic_Perc,Cancer_Chronic_Perc,ObstrPulmonary_Chronic_Perc,Depression_Chronic_Perc,Diabetes_Chronic_Perc,IschemicHeart_Chronic_Perc,Osteoporosis_Chronic_Perc,RheumatoidArthritis_Chronic_Perc,Stroke_Chronic_Perc,HasAllPhysicians_Perc,HasAnyPhysicians_Perc,HasNoPhysicians_Perc,InscClaimAmtReimbursed_Perc,DeductibleAmtPaid_Perc,NoOfMonths_PartACov_Perc,NoOfMonths_PartBCov_Perc,AgeAtService_Perc,PercInsCovered_Perc,ClaimDuration_Perc,ClaimCost_Perc,DailyClaimCost_Perc,IPAnnualReimbursementAmt_Perc,IPAnnualDeductibleAmt_Perc,OPAnnualReimbursementAmt_Perc,OPAnnualDeductibleAmt_Perc,IPDuration_Perc
0,0,PRV51001,0,0.040000,NaN,0.040000,10,19,16,0.240000,0.040000,0.000000,0.0,0.550000,0.000000,0.600000,0.0,5.0,1.0,19.0,1.0,0.400000,0.400000,0.000000,0.400000,0.800000,0.800000,0.200000,0.400000,0.800000,0.800000,0.800000,0.000000,0.600000,0.400000,0.200000,1.000000,0.000000,19400.000000,1068.000000,12.000000,12.000000,77.600000,88.000000,6.000000,20468.000000,4077.400000,77902.000000,2563.200000,1350.000000,236.000000,6.000000
1,0,PRV51001,1,0.040000,NaN,0.040000,10,19,16,0.240000,0.040000,0.000000,0.0,0.550000,0.000000,0.600000,0.0,5.0,1.0,19.0,1.0,0.350000,0.300000,0.000000,0.650000,0.750000,0.650000,0.200000,0.400000,0.250000,0.850000,0.950000,0.300000,0.250000,0.200000,0.050000,1.000000,0.000000,382.000000,0.000000,12.000000,12.000000,77.950000,100.000000,1.550000,382.000000,307.000000,2532.000000,480.600000,2931.500000,520.900000,NaN
2,0,PRV51004,1,0.013423,NaN,0.013423,100,119,112,0.167785,0.013423,0.000000,NaN,0.461538,0.040268,NaN,0.0,NaN,NaN,138.0,9.0,0.338710,0.274194,0.016129,0.516129,0.580645,0.629032,0.112903,0.370968,0.403226,0.790323,0.887097,0.209677,0.306452,0.112903,0.000000,1.000000,0.000000,9241.935484,1068.000000,11.806452,11.806452,69.935484,82.048387,6.161290,10309.935484,2385.064516,12696.612903,1604.064516,2217.741935,724.838710,6.161290
3,0,PRV51007,0,0.027778,NaN,0.027778,48,53,51,0.597222,0.083333,0.027778,0.0,0.420290,0.000000,0.666667,0.0,3.0,1.0,56.0,2.0,0.471429,0.171429,0.000000,0.342857,0.628571,0.357143,0.042857,0.257143,0.414286,0.728571,0.814286,0.285714,0.271429,0.071429,0.057143,1.000000,0.000000,466.714286,1.000000,11.828571,11.928571,68.371429,99.405797,3.357143,467.714286,336.428571,3025.857143,335.657143,3086.000000,748.000000,NaN
4,0,PRV51007,1,0.027778,NaN,0.027778,48,53,51,0.597222,0.083333,0.027778,0.0,0.420290,0.000000,0.666667,0.0,3.0,1.0,56.0,2.0,0.308725,0.154362,0.006711,0.429530,0.590604,0.335570,0.107383,0.275168,0.422819,0.704698,0.724832,0.328859,0.308725,0.114094,0.080537,1.000000,0.000000,350.134228,2.080537,11.865772,11.959732,71.302013,97.847222,2.429530,352.214765,250.362416,4351.879195,434.953020,2194.899329,622.751678,NaN
5,0,PRV51008,0,0.000000,NaN,0.000000,26,31,31,0.255814,0.046512,0.093023,0.0,0.350000,0.024390,0.000000,0.0,2.0,1.0,34.0,2.0,0.438627,0.222318,0.003433,0.365665,0.583691,0.435193,0.141631,0.253219,0.416309,0.685837,0.768240,0.295279,0.284120,0.106438,0.084120,0.998283,0.001717,241.124464,3.175966,11.907296,11.939914,69.567382,98.073386,2.088412,244.300429,196.533906,3623.991416,379.162232,2109.733906,636.328755,NaN
6,0,PRV51008,1,0.000000,NaN,0.000000,26,31,31,0.255814,0.046512,0.093023,0.0,0.350000,0.024390,0.000000,0.0,2.0,1.0,34.0,2.0,0.333333,0.333333,0.000000,0.666667,1.000000,0.333333,0.000000,0.000000,0.666667,1.000000,1.000000,0.000000,0.333333,0.666667,0.000000,1.000000,0.000000,6333.333333,1068.000000,12.000000,12.000000,78.000000,83.000000,6.333333,7401.333333,1255.666667,11710.000000,2136.000000,2413.333333